# Deep Convolutional Neural Network for Limit Order Books

In this notebook, we implement the work of Zhang et al. We further improve this work by adding a self attention layer and by training the model on Order Flow data.

### References:
[1] Ntakaris A, Magris M, Kanniainen J, Gabbouj M, Iosifidis A. Benchmark dataset for mid‐price forecasting of limit order book data with machine learning methods. Journal of Forecasting. 2018 Dec;37(8):852-66. https://arxiv.org/abs/1705.03233

[2] Zhang Z, Zohren S, Roberts S. DeepLOB: Deep convolutional neural networks for limit order books. IEEE Transactions on Signal Processing. 2019 Mar 25;67(11):3001-12. https://arxiv.org/abs/1808.03668

In [1]:
# load packages
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm 
from sklearn.metrics import accuracy_score, classification_report

import torch
import torch.nn.functional as F
from torch.utils import data
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim

## Data Preparation

In [ ]:
import os 
if not os.path.isfile('data.zip'):
    !wget https://github.com/Viccis33/DeepLOB_CNN_Order_Book/raw/refs/heads/master/data.zip
    !unzip -n data.zip
    print('Data downloaded.')
else:
    print('Data already existed.')

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)

def get_label(data):
    lob = data[-5:, :].T
    return lob

def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)

    dY = np.array(Y)

    dataY = dY[T - 1:N]

    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]

    return dataX, dataY

def torch_data(x, y):
    x = torch.from_numpy(x)
    x = torch.unsqueeze(x, 1)
    y = torch.from_numpy(y)
    y = F.one_hot(y, num_classes=3)
    return x, y

In [4]:
class Dataset(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, data, k, num_classes, T):
        """Initialization""" 
        self.k = k
        self.num_classes = num_classes
        self.T = T
            
        x = prepare_x(data)
        y = get_label(data)
        x, y = data_classification(x, y, self.T)
        y = y[:,self.k] - 1
        self.length = len(x)

        x = torch.from_numpy(x)
        self.x = torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index]

We used no auction dataset that is normalised by decimal precision approach in their work. The first seven days are training data and the last three days are testing data. A validation set (20%) from the training set is used to monitor the overfitting behaviours.

The first 40 columns of the FI-2010 dataset are 10 levels ask and bid information for a limit order book and we only use these 40 features in our network. The last 5 columns of the FI-2010 dataset are the labels with different prediction horizons k (10,20,30,50,100).



In [5]:
# please change the data_path to your local path
# data_path = '/nfs/home/zihaoz/limit_order_book/data'

dec_data = np.loadtxt('/kaggle/input/fi-2010/Train_Dst_NoAuction_DecPre_CF_7.txt')
dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]

dec_test1 = np.loadtxt('/kaggle/input/fi-2010/Test_Dst_NoAuction_DecPre_CF_7.txt')
dec_test2 = np.loadtxt('/kaggle/input/fi-2010/Test_Dst_NoAuction_DecPre_CF_8.txt')
dec_test3 = np.loadtxt('/kaggle/input/fi-2010/Test_Dst_NoAuction_DecPre_CF_9.txt')
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))

print(dec_train.shape, dec_val.shape, dec_test.shape)

(149, 203800) (149, 50950) (149, 139587)


In [6]:
batch_size = 32
k1=2

dataset_train = Dataset(data=dec_train, k=k1, num_classes=3, T=100)
dataset_val = Dataset(data=dec_val, k=k1, num_classes=3, T=100)
dataset_test = Dataset(data=dec_test, k=k1, num_classes=3, T=100)

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

print(dataset_train.x.shape, dataset_train.y.shape)

torch.Size([203701, 1, 100, 40]) torch.Size([203701])


In [7]:
tmp_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=1, shuffle=True)

for x, y in tmp_loader:
    print(x)
    print(y)
    print(x.shape, y.shape)
    break

tensor([[[[0.3412, 0.0036, 0.3406,  ..., 0.0010, 0.3385, 0.0010],
          [0.3412, 0.0036, 0.3406,  ..., 0.0010, 0.3390, 0.0032],
          [0.3411, 0.0015, 0.3406,  ..., 0.0094, 0.3390, 0.0032],
          ...,
          [0.3413, 0.0068, 0.3408,  ..., 0.0010, 0.3396, 0.0040],
          [0.3413, 0.0032, 0.3408,  ..., 0.0010, 0.3395, 0.0061],
          [0.3413, 0.0032, 0.3408,  ..., 0.0010, 0.3395, 0.0061]]]],
       dtype=torch.float64)
tensor([1.], dtype=torch.float64)
torch.Size([1, 1, 100, 40]) torch.Size([1])


## Deeplob

In [10]:
class deeplob(nn.Module):
    def __init__(self, y_len):
        super().__init__()
        self.y_len = y_len
        
        # convolution blocks
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.LeakyReLU(negative_slope=0.01),
#             nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,10)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        
        # inception moduels
        self.inp1 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inp2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inp3 = nn.Sequential(
            nn.MaxPool2d((3, 1), stride=(1, 1), padding=(1, 0)),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        
        # lstm layers
        self.lstm = nn.LSTM(input_size=192, hidden_size=64, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(64, self.y_len)

    def forward(self, x):
        # h0: (number of hidden layers, batch size, hidden size)
        h0 = torch.zeros(1, x.size(0), 64).to(device)
        c0 = torch.zeros(1, x.size(0), 64).to(device)
    
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x_inp1 = self.inp1(x)
        x_inp2 = self.inp2(x)
        x_inp3 = self.inp3(x)  
        
        x = torch.cat((x_inp1, x_inp2, x_inp3), dim=1)
        
#         x = torch.transpose(x, 1, 2)
        x = x.permute(0, 2, 1, 3)
        x = torch.reshape(x, (-1, x.shape[1], x.shape[2]))
        
        x, _ = self.lstm(x, (h0, c0))
        x = x[:, -1, :]
        x = self.fc1(x)
        forecast_y = torch.softmax(x, dim=1)
        
        return forecast_y

In [11]:
model = deeplob(y_len = dataset_train.num_classes)
model.to(device)

deeplob(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(1, 2), stride=(1, 2))
    (1): LeakyReLU(negative_slope=0.01)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(4, 1), stride=(1, 1))
    (4): LeakyReLU(negative_slope=0.01)
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(32, 32, kernel_size=(4, 1), stride=(1, 1))
    (7): LeakyReLU(negative_slope=0.01)
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(1, 2), stride=(1, 2))
    (1): Tanh()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(4, 1), stride=(1, 1))
    (4): Tanh()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(32, 32, kernel_size=(4, 1), stride

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
# Function to encapsulate the training loop with early stopping
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs, patience=20):
    
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    best_test_loss = np.inf
    best_test_epoch = 0
    no_improve_epochs = 0  # Counter for early stopping

    for it in tqdm(range(epochs)):
        
        model.train()
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            # Move data to GPU
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())

        # Compute mean train loss
        train_loss = np.mean(train_loss)

        # Evaluate on test set
        model.eval()
        test_loss = []
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)      
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss.append(loss.item())
        test_loss = np.mean(test_loss)

        # Save losses
        train_losses[it] = train_loss
        test_losses[it] = test_loss

        # Check for early stopping
        if test_loss < best_test_loss:
            torch.save(model, './best_val_model_pytorch_30')
            best_test_loss = test_loss
            best_test_epoch = it
            no_improve_epochs = 0  # Reset counter
            print('Model saved (New best validation loss)')
        else:
            no_improve_epochs += 1  # Increment counter

        dt = datetime.now() - t0
        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {test_loss:.4f}, '
              f'Duration: {dt}, Best Val Epoch: {best_test_epoch}')
 
        # **Early stopping condition**
        if no_improve_epochs >= patience:
            print(f'Early stopping triggered after {it+1} epochs. No improvement for {patience} epochs.')
            break  # Stop training

    return train_losses[:it+1], test_losses[:it+1]  # Return only trained epochs


In [14]:
train_losses, val_losses = batch_gd(model, criterion, optimizer, 
                                    train_loader, val_loader, epochs=120)

  1%|          | 1/120 [01:21<2:41:25, 81.39s/it]

Model saved (New best validation loss)
Epoch 1/120, Train Loss: 1.0771, Validation Loss: 1.0753, Duration: 0:01:21.392893, Best Val Epoch: 0


  2%|▏         | 2/120 [02:40<2:37:54, 80.29s/it]

Epoch 2/120, Train Loss: 1.0796, Validation Loss: 1.0756, Duration: 0:01:19.519154, Best Val Epoch: 0


  2%|▎         | 3/120 [03:59<2:35:22, 79.68s/it]

...

Epoch 85/120, Train Loss: 0.7864, Validation Loss: 0.8760, Duration: 0:01:19.787337, Best Val Epoch: 66


 72%|███████▏  | 86/120 [1:53:43<44:45, 78.99s/it]

Epoch 86/120, Train Loss: 0.7858, Validation Loss: 0.8739, Duration: 0:01:18.655831, Best Val Epoch: 66


 72%|███████▏  | 86/120 [1:55:02<45:29, 80.27s/it]

Epoch 87/120, Train Loss: 0.7845, Validation Loss: 0.8748, Duration: 0:01:18.868023, Best Val Epoch: 66
Early stopping triggered after 87 epochs. No improvement for 20 epochs.


In [15]:
model = torch.load('/kaggle/working/best_val_model_pytorch_30')

n_correct = 0.
n_total = 0.
for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    # update counts
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

test_acc = n_correct / n_total
print(f"Test acc: {test_acc:.4f}")

<ipython-input-15-83011c4f4807>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/kaggle/working/best_val_model_pytorch_30')


Test acc: 0.7645


In [16]:
all_targets = []
all_predictions = []

for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)    

In [17]:
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

accuracy_score: 0.7644743633860978
              precision    recall  f1-score   support

           0     0.6980    0.6138    0.6532     31888
           1     0.8061    0.8860    0.8441     78297
           2     0.6969    0.6037    0.6469     29303

    accuracy                         0.7645    139488
   macro avg     0.7337    0.7012    0.7148    139488
weighted avg     0.7584    0.7645    0.7591    139488



## DeepLob Attention

In [18]:
class BahdanauAttention(nn.Module):

    def __init__(self, hidden_dim):
        super().__init__()
        self.encoder_proj = nn.Linear(hidden_dim, hidden_dim)
        self.query_proj   = nn.Linear(hidden_dim, hidden_dim)
        self.score        = nn.Linear(hidden_dim, 1)

    def forward(self, encoder_outputs, query):
     

        query = query.unsqueeze(1)  

        enc_proj = self.encoder_proj(encoder_outputs) 
        q_proj   = self.query_proj(query)             

        energies = torch.tanh(enc_proj + q_proj)

        alignment = self.score(energies)
        attn_weights = F.softmax(alignment.squeeze(-1), dim=1) 

        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs) 
        context = context.squeeze(1) 

        return context, attn_weights

class deeplobAttention(nn.Module):
    def __init__(self, y_len=3, hidden_dim=64, dropout_rate=0.1):
  
        super().__init__()
        self.y_len = y_len
        self.hidden_dim = hidden_dim

        # ========== (1) Convolution Blocks ==========
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, (1, 2), stride=(1, 2)),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, (4, 1)),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, (4, 1)),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(32),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 32, (1, 2), stride=(1, 2)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, (4, 1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, (4, 1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 32, (1, 10)),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, (4, 1)),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, (4, 1)),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(32),
        )

        # ========== (2) Inception Modules ==========
        self.inp1 = nn.Sequential(
            nn.Conv2d(32, 64, (1,1), padding='same'),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, (3,1), padding='same'),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(64),
        )
        self.inp2 = nn.Sequential(
            nn.Conv2d(32, 64, (1,1), padding='same'),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, (5,1), padding='same'),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(64),
        )
        self.inp3 = nn.Sequential(
            nn.MaxPool2d((3,1), stride=(1,1), padding=(1,0)),
            nn.Conv2d(32, 64, (1,1), padding='same'),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(64),
        )

        self.lstm = nn.LSTM(input_size=192, hidden_size=hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)

        self.attn = BahdanauAttention(hidden_dim=hidden_dim)

        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.01),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, y_len),
        )

    def forward(self, x):
        """
        x: shape (B, 1, 100, 40)  # typical for FI-2010
        returns: (B, y_len)  # raw logits
        """
        # 1) CNN stack
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        # 2) Inception
        x_inp1 = self.inp1(x)
        x_inp2 = self.inp2(x)
        x_inp3 = self.inp3(x)
        x = torch.cat([x_inp1, x_inp2, x_inp3], dim=1)  # => (B, 192, T, 1)

        # Prepare for LSTM => (B, T, 192)
        x = x.permute(0, 2, 1, 3)  # => (B, T, 192, 1)
        x = x.reshape(x.size(0), x.size(1), x.size(2))  # => (B, T, 192)

        # 3) LSTM
        lstm_out, (h, c) = self.lstm(x)
        h_final = h.squeeze(0)

        lstm_out = self.dropout(lstm_out)
        h_final  = self.dropout(h_final)

        # 4) Attention
        context, attn_weights = self.attn(lstm_out, h_final)

        # 5) Classifier
        logits = self.fc(context)  # => (B, y_len)
        return logits

In [20]:

def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs, patience=20):
    """
    Train the model using mini-batch gradient descent with early stopping.
    Args:
      model, criterion, optimizer: your usual PyTorch objects
      train_loader, test_loader: data loaders for training and validation
      epochs: max number of epochs to train
      patience: how many epochs to wait before early stopping if no improvement

    Returns:
      train_losses, test_losses: arrays of recorded losses over training
    """
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    best_test_loss = np.inf
    best_test_epoch = 0
    no_improve_epochs = 0

    for it in tqdm(range(epochs)):
        
        model.train()
        t0 = datetime.now()
        batch_train_losses = []
        
        # -- Training --
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Zero gradients
            optimizer.zero_grad()
            # Forward => raw logits
            outputs = model(inputs.float())   # shape (B, y_len)
            # CrossEntropyLoss expects (B, y_len) logits + (B,) class indices
            loss = criterion(outputs, targets.long())
            
            # Backprop + Update
            loss.backward()
            optimizer.step()
            batch_train_losses.append(loss.item())

        # Mean train loss
        train_loss = np.mean(batch_train_losses)

        # -- Validation --
        model.eval()
        batch_test_losses = []
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs.float())
                loss = criterion(outputs, targets.long())
                batch_test_losses.append(loss.item())
        test_loss = np.mean(batch_test_losses)

        # Record
        train_losses[it] = train_loss
        test_losses[it]  = test_loss

        # Early Stopping logic
        if test_loss < best_test_loss:
            torch.save(model, './best_val_model_pytorch_attention_30')
            best_test_loss = test_loss
            best_test_epoch = it
            no_improve_epochs = 0
            print("Model saved (new best validation loss).")
        else:
            no_improve_epochs += 1

        dt = datetime.now() - t0
        print(f"Epoch {it+1}/{epochs} => Train Loss: {train_loss:.4f}, Val Loss: {test_loss:.4f}, "
              f"Duration: {dt}, Best Val Epoch: {best_test_epoch}")

        if no_improve_epochs >= patience:
            print(f"Early stopping after {it+1} epochs (no improvement for {patience} epochs).")
            break

    return train_losses[:it+1], test_losses[:it+1]


model = deeplobAttention(y_len=3, hidden_dim=64, dropout_rate=0.1).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Train
train_losses, val_losses = batch_gd(
    model, criterion, optimizer,
    train_loader=train_loader,
    test_loader=val_loader, 
    epochs=120,
    patience=17
    )


  1%|          | 1/120 [01:26<2:50:57, 86.20s/it]

Model saved (new best validation loss).
Epoch 1/120 => Train Loss: 1.0736, Val Loss: 1.0675, Duration: 0:01:26.194963, Best Val Epoch: 0


  2%|▏         | 2/120 [02:52<2:49:16, 86.07s/it]

Model saved (new best validation loss).
Epoch 2/120 => Train Loss: 1.0699, Val Loss: 1.0668, Duration: 0:01:25.987332, Best Val Epoch: 1


  2%|▎         | 3/120 [04:17<2:47:26, 85.87s/it]

...

Epoch 37/120 => Train Loss: 0.5967, Val Loss: 0.7931, Duration: 0:01:26.717971, Best Val Epoch: 27


 32%|███▏      | 38/120 [54:32<1:58:37, 86.80s/it]

Epoch 38/120 => Train Loss: 0.5946, Val Loss: 0.8109, Duration: 0:01:26.849475, Best Val Epoch: 27


 32%|███▏      | 38/120 [54:50<1:58:21, 86.60s/it]


KeyboardInterrupt: 

In [44]:
model = torch.load('/kaggle/working/best_val_model_pytorch_attention_50')
model.eval() 

all_targets = []
all_predictions = []

# Turn off gradient computations for inference
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.long)

        outputs = model(inputs)  

        _, predictions = torch.max(outputs, dim=1)

        all_targets.append(targets.cpu().numpy())
        all_predictions.append(predictions.cpu().numpy())

all_targets = np.concatenate(all_targets)
all_predictions = np.concatenate(all_predictions)

# Now compute any metrics (accuracy, F1, etc.)
accuracy = np.mean(all_predictions == all_targets)
print(f"Test Accuracy: {accuracy:.4f}")

<ipython-input-44-914ad1ba3651>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/kaggle/working/best_val_model_pytorch_attention_50')


Test Accuracy: 0.7813


In [45]:
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

accuracy_score: 0.781257169075476
              precision    recall  f1-score   support

           0     0.7419    0.7165    0.7290     38408
           1     0.8356    0.8614    0.8483     65996
           2     0.7161    0.7014    0.7087     35084

    accuracy                         0.7813    139488
   macro avg     0.7645    0.7598    0.7620    139488
weighted avg     0.7798    0.7813    0.7803    139488



# DeepLOB with Self Attention on Order Flow data

Our data is in this format: $$ \text{s}_t^{LOB} := (a_t^1, v_t^{1,a}, b_t^1, v_t^{1,b}, ..., a_t^{10}, v_t^{10,a}, b_t^{10}, v_t^{10,b})^T \in \mathbb{R}^{40} $$

where $a_t^i$ and $v_t^{i,a}$ are the $i$-th level ask price and ask volume  
and $b_t^i$ and $v_t^{i,b}$ are the $i$-th level bid price and bid volume  

We will transform it to train our model with Order Flow.
We define the *bid order flows* (bOF) and *ask order flows* (aOF) at a timestamp to be 10-variable vectors computed using two consecutive order book states, where each element is given by

$$ \text{bOF}_{t,i} :=   \left\{
\begin{array}{ll}
      v_t^{i,b}, & b_t^i > b_{t-1}^i \\
      v_t^{i,b} - v_{t-1}^{i,b}, & b_t^i = b_{t-1}^i \\
      -v_t^{i,b}, & b_t^i < b_{t-1}^i \\
\end{array} 
\right. $$

$$ \text{aOF}_{t,i} :=   \left\{
\begin{array}{ll}
      -v_t^{i,a}, & a_t^i > a_{t-1}^i \\
      v_t^{i,a} - v_{t-1}^{i,a}, & a_t^i = a_{t-1}^i \\
      v_t^{i,a}, & a_t^i < a_{t-1}^i \\
\end{array} 
\right. $$

for $i = 1, ..., 10$. With this, we define *order flow* (OF)

$$ \text{OF}_t :=  (\text{bOF}_{t,1}, \text{aOF}_{t,1}, ..., \text{bOF}_{t,10}, \text{aOF}_{t,10})^T \in \mathbb{R}^{20} $$

In [ ]:
def prepare_x(data):
    df1 = data[:40, :].T
    

    # Extract bids, asks, and their volumes
    bids = df1[:, 2::4]  
    bid_volumes = df1[:, 3::4]  
    asks = df1[:, 0::4]  
    ask_volumes = df1[:, 1::4]  

    # Compute changes over time
    bid_changes = np.diff(bids, axis=0, prepend=bids[0:1])
    ask_changes = np.diff(asks, axis=0, prepend=asks[0:1])
    bid_volume_changes = np.diff(bid_volumes, axis=0, prepend=bid_volumes[0:1])
    ask_volume_changes = np.diff(ask_volumes, axis=0, prepend=ask_volumes[0:1])

    # Compute bOF_t
    bOF = np.where(bid_changes > 0, bid_volumes, 
                np.where(bid_changes == 0, bid_volume_changes, 
                            -bid_volumes))

    # Compute aOF_t
    aOF = np.where(ask_changes > 0, -ask_volumes, 
                np.where(ask_changes == 0, ask_volume_changes, 
                            ask_volumes))

    # Concatenate to get OF_t
    OF_t = np.hstack((bOF, aOF))  


    return np.array(OF_t)

def get_label(data):
    lob = data[-5:, :].T
    return lob

def data_classification(X, Y, T):
    
    ############## If you have enough RAM, you can modify the dtype of df, dY and dataX to np.float64 ##############
    
    [N, D] = X.shape
    df = np.array(X, dtype=np.float32)

    dY = np.array(Y, dtype=np.float32)
    dataY = dY[T - 1:N]
    dataX = np.zeros((N - T + 1, T, D), dtype=np.float32)

    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]

    return dataX, dataY

def torch_data(x, y):
    x = torch.from_numpy(x)
    x = torch.unsqueeze(x, 1)
    y = torch.from_numpy(y)
    y = F.one_hot(y, num_classes=3)
    return x, y

In [ ]:
class Dataset(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, data, k, num_classes, T):
        """Initialization""" 
        self.k = k
        self.num_classes = num_classes
        self.T = T
        
        x = prepare_x(data)
        y = get_label(data)
        x, y = data_classification(x, y, self.T)
        y = y[:,self.k] - 1

        self.length = len(x)
        x = torch.from_numpy(x)
        self.x = torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)
        
    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index]

In [ ]:
# please change the data_path to your local path

dec_data = np.loadtxt('Train_Dst_NoAuction_DecPre_CF_7.txt')
dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]

dec_test1 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_7.txt')
dec_test2 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_8.txt')
dec_test3 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_9.txt')
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))

print(dec_train.shape, dec_val.shape, dec_test.shape)

In [ ]:
batch_size = 64

dataset_train = Dataset(data=dec_train, k=4, num_classes=3, T=100)
print("1")
dataset_val = Dataset(data=dec_val, k=4, num_classes=3, T=100)
print("2")
dataset_test = Dataset(data=dec_test, k=4, num_classes=3, T=100)
print("3")

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

print(dataset_train.x.shape, dataset_train.y.shape)

In [ ]:
tmp_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=1, shuffle=True)

for x, y in tmp_loader:
    print(x)
    print(y)
    print(x.shape, y.shape)
    break

In [ ]:
class BahdanauAttention(nn.Module):
  
    def __init__(self, hidden_dim):
        super().__init__()
        self.encoder_proj = nn.Linear(hidden_dim, hidden_dim)
        self.query_proj = nn.Linear(hidden_dim, hidden_dim)
        self.score = nn.Linear(hidden_dim, 1)

    def forward(self, encoder_outputs, query):
        query = query.unsqueeze(1)
        enc_proj = self.encoder_proj(encoder_outputs)  # => (B, seq_len, hidden_dim)
        query_proj = self.query_proj(query)            # => (B, 1, hidden_dim)

     
        energies = torch.tanh(enc_proj + query_proj)
        alignment = self.score(energies)   # => (B, seq_len, 1)

        attn_weights = F.softmax(alignment.squeeze(-1), dim=1)  # => (B, seq_len)

        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs)
        context = context.squeeze(1)

        return context, attn_weights

class DeeplobAttention_OF(nn.Module):
    def __init__(self, y_len=3):
        """
        Args:
          y_len: number of output classes (e.g. 3 for FI-2010 3-label classification).
        """
        super().__init__()
        self.y_len = y_len

        # ========== 1) Convolution Blocks ==========
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        
        #since we use Order Flow data, the dimension is divided by 2 and we don't need the second CNN anymore

        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,10)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )

        # ========== 2) Inception Modules ==========
        self.inp1 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inp2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inp3 = nn.Sequential(
            nn.MaxPool2d((3,1), stride=(1,1), padding=(1,0)),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )

        # ========== 3) LSTM (Encoder) ==========
        # After inception, we have 64 + 64 + 64 = 192 channels
        self.lstm = nn.LSTM(input_size=192, hidden_size=64,
                            num_layers=1, batch_first=True)

        # ========== 4) Bahdanau Attention ==========
        self.attn = BahdanauAttention(hidden_dim=64)

        # ========== 5) Classification Head ==========
        self.fc1 = nn.Linear(64, self.y_len)

    def forward(self, x):
        """
        x: shape (batch_size, 1, k=100, features=some_value),
           typically (B, 1, 100, 40) for the FI-2010 data.
        Returns:
          forecast_y: (B, y_len)  -> (B, 3) for 3-class classification
        """

        x = self.conv1(x)  # => (B, 32, ...)
        # x = self.conv2(x)  # => (B, 32, ...)
        x = self.conv3(x)  # => (B, 32, T, 1)

        # -------------- Step 2: Inception --------------
        x_inp1 = self.inp1(x)  # => (B, 64, T, 1)
        x_inp2 = self.inp2(x)  # => (B, 64, T, 1)
        x_inp3 = self.inp3(x)  # => (B, 64, T, 1)

        x = torch.cat((x_inp1, x_inp2, x_inp3), dim=1)

        x = x.permute(0, 2, 1, 3)          # => (B, T, 192, 1)
        x = x.reshape(x.shape[0], x.shape[1], x.shape[2])  # => (B, T, 192)

        lstm_out, (h, c) = self.lstm(x)    # lstm_out => (B, T, 64)
        final_state = h.squeeze(0)  # => (B, 64)

        context, attn_weights = self.attn(lstm_out, final_state)
        out = self.fc1(context)           # => (B, y_len)
        forecast_y = F.softmax(out, dim=1)  # => (B, y_len)

        return forecast_y

In [ ]:
model = DeeplobAttention_OF(y_len = dataset_train.num_classes)
model.to(device)


In [ ]:
summary(model, (1, 1, 100, 20))

Layer (type:depth-idx)                   Output Shape              Param #
DeeplobAttention_v2                      [1, 3]                    --
├─Sequential: 1-1                        [1, 32, 94, 10]           --
│    └─Conv2d: 2-1                       [1, 32, 100, 10]          96
│    └─LeakyReLU: 2-2                    [1, 32, 100, 10]          --
│    └─BatchNorm2d: 2-3                  [1, 32, 100, 10]          64
│    └─Conv2d: 2-4                       [1, 32, 97, 10]           4,128
│    └─LeakyReLU: 2-5                    [1, 32, 97, 10]           --
│    └─BatchNorm2d: 2-6                  [1, 32, 97, 10]           64
│    └─Conv2d: 2-7                       [1, 32, 94, 10]           4,128
│    └─LeakyReLU: 2-8                    [1, 32, 94, 10]           --
│    └─BatchNorm2d: 2-9                  [1, 32, 94, 10]           64
├─Sequential: 1-2                        [1, 32, 88, 1]            --
│    └─Conv2d: 2-10                      [1, 32, 94, 1]            10,272
│    

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

## Model Training

In [ ]:
# A function to encapsulate the training loop
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs, patience=15):
    
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    best_test_loss = np.inf
    best_test_epoch = 0
    no_improvement_duration = 0 # counter to stop after 10 epochs without improvement on the validation loss

    for it in tqdm(range(epochs)):
        
        model.train()
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            # move data to GPU
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)
            # print("inputs.shape:", inputs.shape)
            # zero the parameter gradients
            optimizer.zero_grad()
            # Forward pass
            # print("about to get model output")
            outputs = model(inputs)
            # print("done getting model output")
            # print("outputs.shape:", outputs.shape, "targets.shape:", targets.shape)
            loss = criterion(outputs, targets)
            # Backward and optimize
            # print("about to optimize")
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
        # Get train loss and test loss
        train_loss = np.mean(train_loss) # a little misleading
    
        model.eval()
        test_loss = []
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)      
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss.append(loss.item())
        test_loss = np.mean(test_loss)

        # Save losses
        train_losses[it] = train_loss
        test_losses[it] = test_loss
        
        if test_loss < best_test_loss:
            torch.save(model, './best_val_model_pytorch')
            best_test_loss = test_loss
            best_test_epoch = it
            no_improvement_duration = 0
            print('model saved')
        else:
            no_improvement_duration +=1

        dt = datetime.now() - t0
        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
          Validation Loss: {test_loss:.4f}, Duration: {dt}, Best Val Epoch: {best_test_epoch}')
        
        if no_improvement_duration == patience:
            print(f"Validation Loss has not improved for {patience} epochs, the gradient descent will be stopped.")
            break

    return train_losses, test_losses

In [ ]:
train_losses, val_losses = batch_gd(model, criterion, optimizer, 
                                    train_loader, val_loader, epochs=70)

  1%|▏         | 1/70 [00:55<1:03:25, 55.16s/it]

model saved
Epoch 1/70, Train Loss: 0.7801,           Validation Loss: 0.7823, Duration: 0:00:55.157618, Best Val Epoch: 0


  3%|▎         | 2/70 [01:49<1:01:40, 54.42s/it]

model saved
Epoch 2/70, Train Loss: 0.7064,           Validation Loss: 0.7624, Duration: 0:00:53.910536, Best Val Epoch: 1


  4%|▍         | 3/70 [02:39<58:53, 52.73s/it]  

...

Epoch 28/70, Train Loss: 0.6415,           Validation Loss: 0.7278, Duration: 0:00:56.571864, Best Val Epoch: 14


 41%|████▏     | 29/70 [26:14<38:56, 56.98s/it]

Epoch 29/70, Train Loss: 0.6407,           Validation Loss: 0.7286, Duration: 0:00:57.285839, Best Val Epoch: 14


 41%|████▏     | 29/70 [27:11<38:26, 56.26s/it]

Epoch 30/70, Train Loss: 0.6404,           Validation Loss: 0.7291, Duration: 0:00:56.733904, Best Val Epoch: 14
Validation Loss has not improved for 15 epochs, the gradient descent will be stopped.


In [ ]:
model = torch.load('best_val_model_pytorch')

n_correct = 0.
n_total = 0.
for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    # update counts
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

test_acc = n_correct / n_total
print(f"Test acc: {test_acc:.4f}")

/tmp/ipykernel_761/1217818421.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('best_val_model_pytorch')


Test acc: 0.8537


In [ ]:
# model = torch.load('best_val_model_pytorch')
all_targets = []
all_predictions = []

for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)    

In [ ]:
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

accuracy_score: 0.853707845836201
              precision    recall  f1-score   support

           0     0.8497    0.8739    0.8616     47915
           1     0.8736    0.8265    0.8494     48050
           2     0.8380    0.8615    0.8496     43523

    accuracy                         0.8537    139488
   macro avg     0.8537    0.8540    0.8535    139488
weighted avg     0.8543    0.8537    0.8536    139488

